## 위성영상을 활용한 선박 탐지 AI 경진대회 제출 스크립트

**주의1: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.**

**주의2: 본 파일의 경로는 제출하시는 모든 모델, 스크립트 구성의 최상위 경로에 위치하고 있어야 합니다.**

- 작성하신 추론용 코드를 본 스크립트 내에 삽입하는 것으로 결과 제출을 수행할 수 있습니다.
- 테스트 데이터가 제공되지 않는 대회로, 안내된 경로를 파라미터로 입력하였을 때 모델이 경로 내의 이미지를 읽어서 추론을 수행할 수 있도록 구성되어야 합니다.

코드는 크게 5가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 설치 
2. 기타 필요한 라이브러리 설치
3. 추론 스크립트 구성
4. aifactory 라이브러리를 이용한 제출 수행
5. 기타 참고사항

※ 가능하면 제출시에는 사용할 모델 및 weight를 제외한 나머지 데이터를 배제하고 제출하는 편을 권장합니다
- 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 및 결과 확인 소요 시간 감소

In [1]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable


### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [2]:
!pip install -U aifactory

Defaulting to user installation because normal site-packages is not writeable


### 2. 기타 필요한 라이브러리 설치
#### 사전 제공되지 않은 라이브러리 가운데 필요한 것이 있는 경우 여기에 설치 명령을 넣습니다
**예)** !pip install tensorflow[and-cuda]      *# PyTorch 대신 GPU를 사용하는 tensorflow를 설치하는 경우*

In [3]:
#!pip install tensorflow[and-cuda]

### 3. 추론 스크립트 구성
#### 추론 스크립트 편집 시 주의사항

1. 전체 추론 실행 코드를 삽입, 테스트셋에 대하여 추론을 수행하고 결과를 지정된 파일명으로 저장하도록 구성
   - 필요한 경우 현재 위치(제목 3.이하, 제목 4.이전)에서 코드를 여러 셀로 나누어 저장해도 무방합니다.
   - 결과 파일은 현재 경로에 **submission.csv**로 저장합니다.
3. 제출 폴더 및 모델 소스코드 내부의 경로는 **./폴더명 또는 ./파일명**으로 **상대 경로**를 지정합니다.
4. 테스트셋 경로는 **/workspace/dataset** 입니다. 
5. 저장할 파일명과 양식에 유의합니다.
   - 대회 페이지 [데이터]탭 참조
   - 파일 양식 가운데 image_name 열은 경로명을 제외하고 정확히 파일명(abcd.jpg)만 들어가야 하므로 코드 작성 시에 참고 부탁드립니다.

1. 이미지 리스트를 불러온다.
2. 리스트에서 이미지 한장을 Image.open()으로 열어본다.
3. 연 이미지를 모델에 맞게 crop한다.
4. crop한 이미지를 텐서로 작성해 버퍼에 저장한다.
5. crop한 뒤 원래 위치를 보정해서 작성하기 위해 원래 이미지에서의 좌상단 위치도 함께 작성해 버퍼에 저장한다.
6. 버퍼에 저장한 crop이미지와 위치를 딕셔너리 형태로 작성한다.

'image_name': 이미지의 원래 이름.png,'image': 이미지 텐서, 'top_left_position':해당 이미지의 좌상단 좌표
데이터로더의 출력은 [batch, 3, crop_size, crop_size]의 이미지 텐서,[batch,1]의 이미지 이름 [batch, 2]의 좌상단 좌표 텐서이다.

In [3]:
import os
from PIL import Image, ImageFile
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import cv2

ImageFile.LOAD_TRUNCATED_IMAGES = True

# 이미지 목록을 가져오는 함수
def get_imglist(dir="./sample/img"):
    imglist = [os.path.join(dir, f).replace("\\", "/") for f in os.listdir(dir) if f.endswith('.png')]
    return imglist

class CroppedImageDataset(Dataset):
    def __init__(self, image_list, crop_size, color_stats_file):
        self.image_list = image_list
        self.crop_size = crop_size
        self.transform = transforms.ToTensor()  # 이미지 -> 텐서 변환
        
        # 색상 통계 정보 불러오기
        color_stats = np.load(color_stats_file)
        self.mean = color_stats['mean']
        self.std = color_stats['std']
        self.color_transform = transforms.Normalize(mean=self.mean.tolist(), std=self.std.tolist())

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        image_name = os.path.basename(image_path)

        # 이미지 열기
        image = Image.open(image_path)
        image_width, image_height = image.size
        # image = np.array(image)
        # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # image = cv2.imread(image_path)
        # if image is not None:
        # image_height, image_width = image.shape[:2]
        

        # 전체 크롭 이미지 개수 계산
        num_crops_x = (image_width + self.crop_size - 1) // self.crop_size
        num_crops_y = (image_height + self.crop_size - 1) // self.crop_size
        total_crops = num_crops_x * num_crops_y

        # 크롭할 영역의 좌상단 좌표를 슬라이딩 윈도우 방식으로 구함
        cropped_images = []
        positions = []
        last_cropped_image_info = None  # 마지막 크롭된 이미지 정보 저장

        for top_left_x in range(0, image_width, self.crop_size):
            for top_left_y in range(0, image_height, self.crop_size):
                # 마지막 부분에서 경계 넘지 않도록 마지막 부분을 맞춤
                bottom_right_x = min(top_left_x + self.crop_size, image_width)
                bottom_right_y = min(top_left_y + self.crop_size, image_height)

                # 이미지 경계 부분에 대해 크롭 영역을 이동시킴
                if bottom_right_x - top_left_x < self.crop_size:
                    top_left_x = image_width - self.crop_size
                    bottom_right_x = image_width

                if bottom_right_y - top_left_y < self.crop_size:
                    top_left_y = image_height - self.crop_size
                    bottom_right_y = image_height

                # 크롭한 이미지 자르기
                # cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
                cropped_image = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x]

                # 크롭한 이미지를 resize
                resize = self.crop_size * 4
                cropped_image = cv2.resize(cropped_image, (resize, resize))

                # 크롭한 이미지를 텐서로 변환
                cropped_image_tensor = self.transform(cropped_image)
                # cropped_image_tensor = self.color_transform(cropped_image_tensor)

                # 크롭한 이미지와 좌상단 좌표 저장
                cropped_images.append(cropped_image_tensor)
                positions.append(torch.tensor([top_left_x, top_left_y]))

                # 마지막 크롭된 이미지의 정보 저장 (좌표와 실제 크기)
                last_cropped_image_info = {
                    'image_tensor': cropped_image_tensor,
                    'top_left': (top_left_x, top_left_y),
                    'bottom_right': (bottom_right_x, bottom_right_y),
                    'size': (bottom_right_x - top_left_x, bottom_right_y - top_left_y)  # 실제 크기 저장
                }

        # 이미지 이름, 크롭한 이미지 텐서 목록, 각 이미지의 좌상단 좌표 및 크롭 개수 반환
        return {
            'image_name': image_name,
            'images': cropped_images,  # 잘라낸 이미지 텐서 리스트
            'top_left_positions': positions,  # 각 이미지의 좌상단 좌표 리스트
            'total_crops': total_crops,  # 총 크롭 이미지 개수
            'last_cropped_image_info': last_cropped_image_info  # 마지막 크롭 이미지 정보
        }

# 배치 데이터를 처리하는 collate_fn 정의
def collate_fn(batch, batch_size):
    all_image_names = []
    all_images = []
    all_top_left_positions = []
    total_crops = 0  # 전체 크롭 이미지 개수를 추적
    last_cropped_images_info = []  # 마지막 크롭 이미지 정보 추적

    for item in batch:
        image_names = [item['image_name']] * len(item['images'])  # 각 이미지에 같은 이름을 붙임
        all_image_names.extend(image_names)
        all_images.extend(item['images'])  # 이미지를 리스트에 추가
        all_top_left_positions.extend(item['top_left_positions'])  # 좌상단 좌표 추가
        total_crops += item['total_crops']  # 총 크롭 개수 계산
        last_cropped_images_info.append(item['last_cropped_image_info'])  # 마지막 크롭 정보 추가

    # 전체 이미지 목록을 batch_size 크기씩 나눠서 반환
    batch_start = 0
    while batch_start < len(all_images):
        images_batch = torch.stack(all_images[batch_start:batch_start + batch_size])  # batch_size만큼 이미지 묶기
        positions_batch = torch.stack(all_top_left_positions[batch_start:batch_start + batch_size])  # batch_size만큼 좌표 묶기
        names_batch = all_image_names[batch_start:batch_start + batch_size]  # batch_size만큼 이미지 이름 묶기
        
        batch_start += batch_size
        
        yield {
            'image_names': names_batch,  # 이미지 이름 리스트
            'images': images_batch,  # [batch_size, 3, crop_size, crop_size]
            'top_left_positions': positions_batch,  # [batch_size, 2]
            'total_crops': total_crops,  # 전체 크롭 이미지 개수
            'last_cropped_images_info': last_cropped_images_info  # 마지막 크롭 이미지 정보
        }

# dataset = CroppedImageDataset(img_list, crop_size)

# # DataLoader에서 batch_size를 16으로 설정
# batch_size = 32
# dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=lambda x: collate_fn(x, batch_size))

# # 배치 데이터 확인 및 검증
# for batch in dataloader:
#     total_images = 0  # 전체 크롭 이미지 개수를 추적
#     for sub_batch in batch:  # collate_fn이 배치 크기만큼 나눠서 반환
#         print(f"Batch size: {len(sub_batch['image_names'])}")  # 배치 내 이미지 개수 확인
#         print(f"Image Tensor Shape: {sub_batch['images'].shape}")  # [배치 크기, 3, crop_size, crop_size]
#         print(f"Image Names: {sub_batch['image_names']}")  # 이미지 이름 리스트
#         print(f"Top Left Positions Shape: {sub_batch['top_left_positions'].shape}")  # [배치 크기, 2]

#         # 마지막 크롭된 이미지 정보 확인
#         for last_info in sub_batch['last_cropped_images_info']:
#             print(f"Last Cropped Image Top-Left: {last_info['top_left']}")
#             print(f"Last Cropped Image Bottom-Right: {last_info['bottom_right']}")
#             print(f"Last Cropped Image Size: {last_info['size']}")  # 마지막 크롭된 이미지 크기 확인

#             # 크롭된 이미지가 정확한 크기인지 확인 (경계 부분이 잘 처리되었는지 확인)
#             if last_info['size'][0] <= crop_size and last_info['size'][1] <= crop_size:
#                 print("Last cropped image size is correct.")
#             else:
#                 print("Last cropped image size is incorrect.")

#         total_images += len(sub_batch['image_names'])  # 전체 이미지 개수 증가

#     # 총 크롭 이미지 개수와 배치에서 나온 이미지 개수 비교
#     print(f"Total cropped images (from dataset): {sub_batch['total_crops']}")
#     print(f"Total images processed from batches: {total_images}")
    
#     # 총 크롭 이미지 개수가 배치에서 모두 나왔는지 검증
#     if total_images == sub_batch['total_crops']:
#         print("All cropped images from the dataset have been processed correctly.")
#     else:
#         print(f"Discrepancy: Processed {total_images} images, but expected {sub_batch['total_crops']} images.")
    
#     break  # 첫 번째 이미지 데이터만 확인


In [ ]:
from ultralytics import YOLO
import torch

# 이미지 목록을 가져오는 함수
test_path = '../'
# 사용 예시

crop_size = 256  # 크롭할 이미지의 크기
# test_path = '/workspace/dataset'
img_list = get_imglist(test_path)
color_stats_file = './train_color_stats.npz'
batch_size = 1024
# 모델 정의 
model = YOLO("./best.pt")  # YOLO OBB 모델 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "mps")  # GPU 또는 MPS 사용

# 모델은 이미 내부적으로 GPU/MPS를 사용하므로 입력 이미지를 device로 보냄
dataset = CroppedImageDataset(img_list, crop_size, color_stats_file)

# DataLoader에서 batch_size를 16으로 설정
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=lambda x: collate_fn(x, batch_size))

# YOLO 모델을 사용한 예측 함수
def run_yolo_on_images(dataloader, model, device):
    results = []  # 예측 결과를 저장할 리스트
    
    for batch in dataloader:
        for sub_batch in batch:  # 각 sub_batch에 대해 처리
            images = sub_batch['images'].to(device)  # 이미지를 device로 전송
            top_left_positions = sub_batch['top_left_positions'].to(device)  # 좌상단 좌표도 device로 전송
            
            # YOLO 모델 예측 수행
            preds = model.predict(images,conf=0.1,save=True)  # Ultralytics YOLO 모델의 predict 함수 사용

            obb = [ pred.obb.xywhr for pred in preds]
            results.append({
                'image_names': sub_batch['image_names'],
                'obb':obb,
                'top_left_positions': top_left_positions
                
            })
            print(f"Processed {len(sub_batch['image_names'])} images with predictions.")
    
    return results

# 모델을 사용한 예측 수행
predictions = []
predictions += run_yolo_on_images(dataloader, model, device)



WARNING  torch.Tensor inputs should be normalized 0.0-1.0 but max value is 6.543742656707764. Dividing input by 255.

WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



In [3]:
import csv
import torch
from torchvision.ops import nms
import math

# 파일 저장할 CSV 경로
csv_file = "./submission.csv"

# 데이터 샘플 (image_name, cx, cy, width, height, angle 등)
data = []

# NMS 임계값 (IoU 임계값)
nms_threshold = 0.7

# 'predictions' 리스트에 있는 각 배치에서 데이터를 추출
for i in range(len(predictions)):  # predictions 리스트에서 하나씩 꺼냄
    image_name = predictions[i]['image_names']  # 각 배치의 이미지 이름 리스트
    obb_list = predictions[i]['obb']  # 각 배치의 obb 리스트
    top_left_pos_list = predictions[i]['top_left_positions']  # 각 배치의 top_left_positions 리스트

    # 각 배치에서 이미지별로 순회
    for j in range(len(obb_list)):
        obb_tensor = obb_list[j]
        top_left_pos = top_left_pos_list[j]

        # obb_tensor가 비어있지 않은 경우에만 처리
        if len(obb_tensor) > 0:
            # NMS 처리를 위한 준비
            boxes = []
            scores = []  # NMS를 위해 점수가 필요 (여기서는 객체 너비를 임시 점수로 사용)
            for k in range(len(obb_tensor)):
                # cx = obb_tensor[k][0].item() + top_left_pos[0].item()
                # cy = obb_tensor[k][1].item() + top_left_pos[1].item()
                cx = obb_tensor[k][0].item()/4.0 + top_left_pos[0].item()
                cy = obb_tensor[k][1].item()/4.0 + top_left_pos[1].item()
                width = obb_tensor[k][2].item()/4.0
                height = obb_tensor[k][3].item()/4.0
                angle = obb_tensor[k][4].item()

                # 사각형 좌표로 변환 (cx, cy, width, height -> x1, y1, x2, y2)
                x1 = cx - width / 2
                y1 = cy - height / 2
                x2 = cx + width / 2
                y2 = cy + height / 2

                # 박스와 점수 추가
                boxes.append([x1, y1, x2, y2])
                scores.append(width)  # width를 임시 점수로 사용

            # NMS 수행
            boxes_tensor = torch.tensor(boxes, dtype=torch.float32)
            scores_tensor = torch.tensor(scores, dtype=torch.float32)
            nms_indices = nms(boxes_tensor, scores_tensor, nms_threshold)

            # NMS 후 남은 객체들에 대해 각도 변환 및 데이터 추가
            # NMS 후 남은 객체들에 대해 각도 변환 및 데이터 추가
            for idx in nms_indices:
                cx = obb_tensor[idx][0].item()/4.0 + top_left_pos[0].item()
                cy = obb_tensor[idx][1].item()/4.0 + top_left_pos[1].item()
                width = obb_tensor[idx][2].item()/4.0
                height = obb_tensor[idx][3].item()/4.0
                # cx = round(obb_tensor[idx][0].item()/4.0 + top_left_pos[0].item(), 2)
                # cy = round(obb_tensor[idx][1].item()/4.0 + top_left_pos[1].item(), 2)
                # width = round(obb_tensor[idx][2].item()/4.0, 2)
                # height = round(obb_tensor[idx][3].item()/4.0, 2)
                angle = obb_tensor[idx][4].item()
                
                # 라디안을 도 단위로 변환
                angle_deg = math.degrees(angle)
                
                # 각도를 0~360도 범위로 변환
                if angle_deg < 0:
                    angle_deg += 360
                    
                angle_deg = round(angle_deg, 2)

                # 데이터 추가
                data.append([image_name[j], cx, cy, width, height, angle_deg])
# CSV 파일로 저장
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSV의 헤더 작성
    writer.writerow(['image_name', 'cx', 'cy', 'width', 'height', 'angle'])
    
    # 각 행을 작성
    writer.writerows(data)

print(f"CSV 파일 '{csv_file}'이(가) 성공적으로 생성되었습니다.")


CSV 파일 './submission.csv'이(가) 성공적으로 생성되었습니다.


### 4. aifactory 라이브러리를 이용한 제출 수행
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 플랫폼 우측 상단 아이콘 - [마이페이지] - [활동히스토리] 아래 [Competition] 란에서 대회 이름으로 확인하실 수 있습니다.

In [6]:
print('n ice')

n ice


In [1]:
import aifactory.score as aif
import time

t = time.time()
aif.submit(model_name="yolo",
           key="128fd22e-34e1-4e7a-b9c9-3423c2e859ce")
print("time:", time.time() - t)

file : task.py
python
제출 완료
time: 19.929635047912598


### 5. 기타 참고사항
- 추론 수행 시간:
  - 일반적으로 기본 사이즈의 YOLO계열 모델 사용 시 test set 전체 추론에는 1시간 정도가 소요됩니다.
- CUDA Out of Memory 문제:
  - GPU OOM이 발생하는 경우 
    - 각 image 사이 또는 batch 사이에 torch.cuda.empty_cache() 및 gc.collect()를 입력하여 VRAM의 낭비 공간을 정리하거나
    - Batch size를 조절하는 방법 등을 활용해볼 수 있습니다.
- Storage:
  - 추론 환경에서는 참가자 분의 모델 및 기타 산출물이 임시 저장되는 공간으로 기본 32GB가 제공되므로 작업 시에 참고 부탁드립니다.